In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import random
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Load and prepare the dataset
df = pd.read_csv('epi_r.csv')

# Enhanced data preparation with expanded dietary keywords
def prepare_data(df):
    """Prepare and clean the dataset with more features"""
    
    # Select relevant columns
    required_columns = ['title', 'rating', 'calories', 'protein', 'fat', 'sodium']
    
    # Check for missing required columns and handle gracefully
    available_columns = [col for col in required_columns if col in df.columns]
    if len(available_columns) < len(required_columns):
        print(f"Warning: Some required columns are missing. Using: {available_columns}")
    
    # Create clean dataframe
    clean_df = df[available_columns].copy()
    
    # Handle missing values
    clean_df = clean_df.dropna()
    
    # Remove unrealistic values
    clean_df = clean_df[
        (clean_df['calories'] > 50) & 
        (clean_df['calories'] < 2000) &
        (clean_df['protein'] >= 0) &
        (clean_df['fat'] >= 0)
    ]
    
    # Add meal-time features (extracted from title)
    breakfast_keywords = ['breakfast', 'pancake', 'waffle', 'cereal', 'oatmeal', 'muffin', 
                         'scrambled', 'omelet', 'frittata', 'granola', 'smoothie', 'bagel',
                         'croissant', 'danish', 'morning', 'brunch']
    
    lunch_keywords = ['sandwich', 'salad', 'soup', 'wrap', 'panini', 'pita', 'taco',
                     'burrito', 'quesadilla', 'lunch', 'midday', 'noodle bowl']
    
    dinner_keywords = ['dinner', 'roast', 'steak', 'pasta', 'casserole', 'curry',
                      'stir-fry', 'grilled', 'baked', 'braised', 'stew', 'chili',
                      'lasagna', 'risotto', 'paella', 'supper', 'evening']
    
    # Calculate meal type scores
    clean_df['breakfast_score'] = clean_df['title'].str.lower().apply(
        lambda x: sum(1 for word in breakfast_keywords if word in x))
    clean_df['lunch_score'] = clean_df['title'].str.lower().apply(
        lambda x: sum(1 for word in lunch_keywords if word in x))
    clean_df['dinner_score'] = clean_df['title'].str.lower().apply(
        lambda x: sum(1 for word in dinner_keywords if word in x))
    
    # Improved meal type assignment
    def assign_meal_type(row):
        scores = {
            'breakfast': row['breakfast_score'] * 2 + (2 if row['calories'] < 400 else 0),
            'lunch': row['lunch_score'] * 2 + (2 if 400 <= row['calories'] <= 700 else 0),
            'dinner': row['dinner_score'] * 2 + (2 if row['calories'] > 700 else 0)
        }
        
        # If no clear winner based on keywords, use calories
        if max(scores.values()) == 0:
            if row['calories'] < 400:
                return 'breakfast'
            elif row['calories'] < 700:
                return 'lunch'
            else:
                return 'dinner'
        
        return max(scores, key=scores.get)
    
    clean_df['meal_type'] = clean_df.apply(assign_meal_type, axis=1)
    
    # Enhanced dietary classification
    # Expanded vegetarian keywords
    meat_keywords = ['chicken', 'beef', 'pork', 'fish', 'lamb', 'turkey', 'bacon', 'meat',
                    'sausage', 'ham', 'steak', 'salmon', 'tuna', 'shrimp', 'lobster',
                    'crab', 'duck', 'veal', 'venison', 'prosciutto', 'chorizo', 'seafood',
                    'anchovy', 'sardine', 'halibut', 'cod', 'tilapia', 'meatball', 'ribs']
    
    # Check for vegetarian column or create based on keywords
    if 'vegetarian' in df.columns:
        clean_df['vegetarian'] = df.loc[clean_df.index, 'vegetarian'].fillna(False)
    else:
        clean_df['vegetarian'] = ~clean_df['title'].str.lower().apply(
            lambda x: any(meat in x for meat in meat_keywords))
    
    # Expanded gluten-free keywords
    gluten_keywords = ['pasta', 'bread', 'flour', 'wheat', 'noodle', 'cake', 'cookie',
                      'muffin', 'croissant', 'bagel', 'pizza', 'pie crust', 'pastry',
                      'cracker', 'cereal', 'barley', 'rye', 'couscous', 'bulgur',
                      'semolina', 'spelt', 'farro', 'panko', 'breadcrumb', 'biscuit',
                      'pretzel', 'waffle', 'pancake', 'donut', 'brioche', 'baguette']
    
    # Check for gluten-free column or create based on keywords
    if 'gluten-free' in df.columns:
        clean_df['gluten_free'] = df.loc[clean_df.index, 'gluten-free'].fillna(False)
    elif 'wheat/gluten-free' in df.columns:
        clean_df['gluten_free'] = df.loc[clean_df.index, 'wheat/gluten-free'].fillna(False)
    else:
        clean_df['gluten_free'] = ~clean_df['title'].str.lower().apply(
            lambda x: any(gluten in x for gluten in gluten_keywords))
    
    # Ensure boolean types
    clean_df['vegetarian'] = clean_df['vegetarian'].astype(bool)
    clean_df['gluten_free'] = clean_df['gluten_free'].astype(bool)
    
    return clean_df

# Prepare data
clean_df = prepare_data(df)
print(f"Prepared dataset shape: {clean_df.shape}")
print(f"Vegetarian recipes: {clean_df['vegetarian'].sum()}")
print(f"Gluten-free recipes: {clean_df['gluten_free'].sum()}")

In [ ]:
# Optimized neural network for meal type classification
def build_meal_classifier(input_dim):
    """Build an optimized neural network for meal type classification"""
    
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(3, activation='softmax')  # 3 classes: breakfast, lunch, dinner
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Train meal type classifier
def train_meal_classifier(clean_df):
    """Train the meal type classification model"""
    
    # Prepare features
    feature_cols = ['calories', 'protein', 'fat', 'sodium', 'rating', 
                   'breakfast_score', 'lunch_score', 'dinner_score']
    
    # Ensure all features exist
    available_features = [col for col in feature_cols if col in clean_df.columns]
    X = clean_df[available_features].values
    
    # Prepare labels
    le = LabelEncoder()
    y = le.fit_transform(clean_df['meal_type'])
    y_categorical = keras.utils.to_categorical(y)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Build and train model with early stopping
    model = build_meal_classifier(X_train.shape[1])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(
        X_train_scaled, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate
    test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
    print(f"Meal classifier accuracy: {test_acc:.3f}")
    
    return model, scaler, le, available_features

# Train the classifier
meal_classifier, meal_scaler, meal_label_encoder, meal_features = train_meal_classifier(clean_df)


Meal classifier accuracy: 0.996


In [ ]:
# Optimized neural network for meal compatibility scoring
def build_compatibility_scorer():
    """Build an optimized neural network to score compatibility between meals"""
    
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(10,)),  # Adjusted input size
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output: compatibility score 0-1
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Generate training data for compatibility scorer
def generate_compatibility_data(clean_df, n_samples=20000):
    """Generate synthetic training data for meal compatibility"""
    
    X = []
    y = []
    
    for _ in range(n_samples):
        # Randomly select two meals
        idx1, idx2 = np.random.choice(len(clean_df), 2, replace=False)
        meal1 = clean_df.iloc[idx1]
        meal2 = clean_df.iloc[idx2]
        
        # Create feature vector
        features = np.array([
            meal1['calories'], meal1['protein'], meal1['fat'], meal1['rating'],
            meal1['meal_type'] == 'breakfast',
            meal2['calories'], meal2['protein'], meal2['fat'], meal2['rating'],
            meal2['meal_type'] == 'dinner'
        ])
        
        # Create compatibility score (enhanced heuristic)
        compatibility = 0
        
        # Different meal types is good
        if meal1['meal_type'] != meal2['meal_type']:
            compatibility += 0.4
        
        # Complementary nutrition
        total_calories = meal1['calories'] + meal2['calories']
        if 800 <= total_calories <= 1400:
            compatibility += 0.3
        
        # Balanced protein
        total_protein = meal1['protein'] + meal2['protein']
        if 25 <= total_protein <= 60:
            compatibility += 0.2
        
        # Similar ratings
        if abs(meal1['rating'] - meal2['rating']) < 0.5:
            compatibility += 0.1
        
        X.append(features)
        y.append(compatibility)
    
    return np.array(X), np.array(y)

# Train compatibility scorer
X_compat, y_compat = generate_compatibility_data(clean_df)
X_train, X_test, y_train, y_test = train_test_split(
    X_compat, y_compat, test_size=0.2, random_state=42)

compatibility_model = build_compatibility_scorer()

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

compatibility_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

test_loss, test_mae = compatibility_model.evaluate(X_test, y_test, verbose=0)
print(f"Compatibility model MAE: {test_mae:.3f}")

Compatibility model MAE: 0.084


In [ ]:
# Enhanced optimization neural network for daily meal selection
class MealPlanOptimizer:
    """Neural network-based meal plan optimizer"""
    
    def __init__(self, clean_df, meal_classifier, compatibility_model):
        self.clean_df = clean_df
        self.meal_classifier = meal_classifier
        self.compatibility_model = compatibility_model
        
    def score_meal_day(self, breakfast, lunch, dinner):
        """Score a complete day of meals"""
        
        # Calculate nutritional totals
        total_calories = breakfast['calories'] + lunch['calories'] + dinner['calories']
        total_protein = breakfast['protein'] + lunch['protein'] + dinner['protein']
        total_fat = breakfast['fat'] + lunch['fat'] + dinner['fat']
        avg_rating = (breakfast['rating'] + lunch['rating'] + dinner['rating']) / 3
        
        # Score components (improved scoring)
        calorie_score = np.exp(-((total_calories - 2000) / 300) ** 2)  # Gaussian around 2000
        protein_score = np.exp(-((total_protein - 75) / 25) ** 2)      # Gaussian around 75g
        fat_score = 1.0 if 50 <= total_fat <= 80 else 0.5
        rating_score = avg_rating / 5.0
        
        # Variety score (penalize if same meal type appears twice)
        variety_score = 1.0
        
        # Overall score
        overall_score = (calorie_score * 0.3 + protein_score * 0.25 + 
                        fat_score * 0.15 + rating_score * 0.2 + variety_score * 0.1)
        
        return overall_score
    
    def calculate_compatibility(self, meal1, meal2):
        """Calculate compatibility between two meals"""
        
        features = np.array([[
            meal1.calories, meal1.protein, meal1.fat, meal1.rating,
            meal1.meal_type == 'breakfast',
            meal2.calories, meal2.protein, meal2.fat, meal2.rating,
            meal2.meal_type == 'dinner'
        ]])
        
        return self.compatibility_model.predict(features, verbose=0)[0][0]
    
    def generate_meal_plan(self, num_days=3, vegetarian=False, gluten_free=False):
        """Generate optimized meal plan using neural networks"""
        
        # Filter recipes based on dietary restrictions
        filtered_df = self.clean_df.copy()
        if vegetarian:
            filtered_df = filtered_df[filtered_df['vegetarian'] == True]
        if gluten_free:
            filtered_df = filtered_df[filtered_df['gluten_free'] == True]
        
        # Check if we have enough recipes
        meal_types = ['breakfast', 'lunch', 'dinner']
        for meal_type in meal_types:
            count = len(filtered_df[filtered_df['meal_type'] == meal_type])
            if count < 5:
                print(f"Warning: Only {count} {meal_type} options available")
        
        meal_plan = []
        used_recipes = set()  # Track used recipes to avoid repetition
        
        for day in range(num_days):
            best_score = -1
            best_meals = None
            
            # Try multiple combinations
            for _ in range(30):  # Reduced iterations for speed
                # Select candidates avoiding previously used recipes
                available_df = filtered_df[~filtered_df.index.isin(used_recipes)]
                
                if len(available_df) < 3:
                    available_df = filtered_df  # Reset if not enough unique recipes
                
                try:
                    breakfast_options = available_df[available_df['meal_type'] == 'breakfast']
                    lunch_options = available_df[available_df['meal_type'] == 'lunch']
                    dinner_options = available_df[available_df['meal_type'] == 'dinner']
                    
                    if len(breakfast_options) == 0 or len(lunch_options) == 0 or len(dinner_options) == 0:
                        print(f"Warning: Not enough recipes for day {day + 1}")
                        continue
                    
                    # Sample meals
                    b = breakfast_options.sample(1).iloc[0]
                    l = lunch_options.sample(1).iloc[0]
                    d = dinner_options.sample(1).iloc[0]
                    
                    # Calculate compatibility scores
                    bl_compat = self.calculate_compatibility(b, l)
                    ld_compat = self.calculate_compatibility(l, d)
                    bd_compat = self.calculate_compatibility(b, d)
                    
                    # Overall day score
                    day_score = self.score_meal_day(
                        {'calories': b.calories, 'protein': b.protein, 'fat': b.fat, 'rating': b.rating},
                        {'calories': l.calories, 'protein': l.protein, 'fat': l.fat, 'rating': l.rating},
                        {'calories': d.calories, 'protein': d.protein, 'fat': d.fat, 'rating': d.rating}
                    )
                    
                    # Combined score
                    combo_score = day_score * 0.6 + (bl_compat + ld_compat + bd_compat) / 3 * 0.4
                    
                    if combo_score > best_score:
                        best_score = combo_score
                        best_meals = (b, l, d)
                
                except Exception as e:
                    print(f"Error in meal selection: {e}")
                    continue
            
            # Add best combination to meal plan
            if best_meals:
                b, l, d = best_meals
                used_recipes.add(b.name)
                used_recipes.add(l.name)
                used_recipes.add(d.name)
                
                meal_plan.append({
                    'day': day + 1,
                    'meals': {
                        'breakfast': {
                            'title': b.title, 'calories': b.calories, 
                            'protein': b.protein, 'fat': b.fat, 'rating': b.rating
                        },
                        'lunch': {
                            'title': l.title, 'calories': l.calories,
                            'protein': l.protein, 'fat': l.fat, 'rating': l.rating
                        },
                        'dinner': {
                            'title': d.title, 'calories': d.calories,
                            'protein': d.protein, 'fat': d.fat, 'rating': d.rating
                        }
                    },
                    'total_calories': int(b.calories + l.calories + d.calories),
                    'total_protein': round(b.protein + l.protein + d.protein, 1),
                    'total_fat': round(b.fat + l.fat + d.fat, 1),
                    'optimization_score': round(best_score, 3)
                })
        
        return meal_plan

# Create the optimizer
optimizer = MealPlanOptimizer(clean_df, meal_classifier, compatibility_model)

In [ ]:
# Enhanced display and analysis functions
def display_optimized_meal_plan(meal_plan):
    """Display optimized meal plan with enhanced formatting"""
    
    for day_plan in meal_plan:
        print(f"\n{'='*50}")
        print(f"Day {day_plan['day']} | Score: {day_plan['optimization_score']:.3f}")
        print(f"{'='*50}")
        print(f"Total: {day_plan['total_calories']} cal | {day_plan['total_protein']}g protein | {day_plan['total_fat']}g fat")
        print(f"{'-'*50}")
        
        for meal_type, meal_info in day_plan['meals'].items():
            print(f"\n{meal_type.upper()}: {meal_info['title']}")
            print(f"  {meal_info['calories']} cal | {meal_info['protein']}g protein | {meal_info['fat']}g fat")

def analyze_meal_plan_quality(meal_plan):
    """Enhanced analysis of nutritional quality"""
    
    daily_calories = []
    daily_protein = []
    daily_fat = []
    optimization_scores = []
    
    for day in meal_plan:
        daily_calories.append(day['total_calories'])
        daily_protein.append(day['total_protein'])
        daily_fat.append(day['total_fat'])
        optimization_scores.append(day['optimization_score'])
    
    print("\n" + "="*50)
    print("MEAL PLAN ANALYSIS")
    print("="*50)
    print(f"Average Daily Calories: {np.mean(daily_calories):.0f} ± {np.std(daily_calories):.0f}")
    print(f"Average Daily Protein: {np.mean(daily_protein):.1f}g ± {np.std(daily_protein):.1f}g")
    print(f"Average Daily Fat: {np.mean(daily_fat):.1f}g ± {np.std(daily_fat):.1f}g")
    print(f"Average Optimization Score: {np.mean(optimization_scores):.3f}")
    
    # Nutritional guidelines check
    meets_calories = all(1600 <= cal <= 2400 for cal in daily_calories)
    meets_protein = all(45 <= prot <= 100 for prot in daily_protein)
    meets_fat = all(45 <= fat <= 80 for fat in daily_fat)
    
    print(f"\n✓ Meets Calorie Guidelines (1600-2400): {'Yes' if meets_calories else 'No'}")
    print(f"✓ Meets Protein Guidelines (45-100g): {'Yes' if meets_protein else 'No'}")
    print(f"✓ Meets Fat Guidelines (45-80g): {'Yes' if meets_fat else 'No'}")
    print("="*50)

# Generate optimized meal plans
print("STANDARD OPTIMIZED MEAL PLAN")
standard_plan = optimizer.generate_meal_plan(num_days=3)
display_optimized_meal_plan(standard_plan)
analyze_meal_plan_quality(standard_plan)

print("\n\nVEGETARIAN OPTIMIZED MEAL PLAN")
vegetarian_plan = optimizer.generate_meal_plan(num_days=3, vegetarian=True)
display_optimized_meal_plan(vegetarian_plan)
analyze_meal_plan_quality(vegetarian_plan)

print("\n\nGLUTEN-FREE OPTIMIZED MEAL PLAN")
gluten_free_plan = optimizer.generate_meal_plan(num_days=3, gluten_free=True)
display_optimized_meal_plan(gluten_free_plan)
analyze_meal_plan_quality(gluten_free_plan)


STANDARD OPTIMIZED MEAL PLAN

Day 1 | Score: 0.691
Total: 2258 cal | 79.0g protein | 64.0g fat
--------------------------------------------------

BREAKFAST: Zucchini Fritters 
  257.0 cal | 5.0g protein | 20.0g fat

LUNCH: Grilled Tuna and Watercress Salad with Asian Flavors 
  424.0 cal | 31.0g protein | 26.0g fat

DINNER: Pizza Dough 
  1577.0 cal | 43.0g protein | 18.0g fat

Day 2 | Score: 0.778
Total: 1878 cal | 81.0g protein | 141.0g fat
--------------------------------------------------

BREAKFAST: Strawberry-Topped Lime Mousse Tart 
  387.0 cal | 5.0g protein | 21.0g fat

LUNCH: Pepper-Crusted Beef, Bacon, and Arugula Sandwiches with Spicy Mustard 
  763.0 cal | 35.0g protein | 66.0g fat

DINNER: Pork Fricassée with Mushrooms and Carrots 
  728.0 cal | 41.0g protein | 54.0g fat

Day 3 | Score: 0.775
Total: 1974 cal | 83.0g protein | 144.0g fat
--------------------------------------------------

BREAKFAST: Shoe String Potatoes (Pommes Pailles) 
  360.0 cal | 4.0g protein | 26.0g

In [ ]:
# Easy-to-use wrapper function
def create_optimized_meal_plan(days=3, vegetarian=False, gluten_free=False):
    """
    Create an optimized meal plan using neural networks
    
    Parameters:
    - days: Number of days in the plan (default 3)
    - vegetarian: Boolean for vegetarian restriction
    - gluten_free: Boolean for gluten-free restriction
    
    Returns:
    - meal_plan: Optimized meal plan
    """
    
    print(f"\nGenerating {days}-day meal plan...")
    if vegetarian:
        print("Dietary restriction: Vegetarian")
    if gluten_free:
        print("Dietary restriction: Gluten-free")
    
    meal_plan = optimizer.generate_meal_plan(
        num_days=days,
        vegetarian=vegetarian,
        gluten_free=gluten_free
    )
    
    display_optimized_meal_plan(meal_plan)
    analyze_meal_plan_quality(meal_plan)
    
    return meal_plan

# Example usage
print("\n" + "="*50)
print("QUICK START EXAMPLES")
print("="*50)

# Create default 3-day plan
default_plan = create_optimized_meal_plan()

# Create 3-day vegetarian plan
vegetarian_plan = create_optimized_meal_plan(vegetarian=True)

# Create 3-day gluten-free plan  
gluten_free_plan = create_optimized_meal_plan(gluten_free=True)

# Create 3-day vegetarian and gluten-free plan
veg_gf_plan = create_optimized_meal_plan(vegetarian=True, gluten_free=True)


QUICK START EXAMPLES

Generating 3-day meal plan...

Day 1 | Score: 0.741
Total: 1970 cal | 67.0g protein | 116.0g fat
--------------------------------------------------

BREAKFAST: Pastry Dough 
  230.0 cal | 3.0g protein | 16.0g fat

LUNCH: Sticky Date and Almond Bread Pudding with Amaretto Zabaglione 
  568.0 cal | 10.0g protein | 27.0g fat

DINNER: Braised Chicken and Rice with Orange, Saffron, Almond, and Pistachio Syrup 
  1172.0 cal | 54.0g protein | 73.0g fat

Day 2 | Score: 0.790
Total: 1909 cal | 76.0g protein | 135.0g fat
--------------------------------------------------

BREAKFAST: Roasted Cauliflower with Onions and Fennel 
  205.0 cal | 4.0g protein | 14.0g fat

LUNCH: Roast New York Strip Loin with Adobo Rub 
  569.0 cal | 44.0g protein | 41.0g fat

DINNER: Braised Spiced Pork with Cao Lau Noodles 
  1135.0 cal | 28.0g protein | 80.0g fat

Day 3 | Score: 0.798
Total: 1968 cal | 93.0g protein | 61.0g fat
--------------------------------------------------

BREAKFAST: Pru

In [ ]:
# Standard 3-day plan
plan = create_optimized_meal_plan()

# Vegetarian plan
veg_plan = create_optimized_meal_plan(vegetarian=True)

# Custom 5-day gluten-free plan
gf_plan = create_optimized_meal_plan(days=5, gluten_free=True)


Generating 3-day meal plan...

Day 1 | Score: 0.728
Total: 1843 cal | 55.0g protein | 146.0g fat
--------------------------------------------------

BREAKFAST: Veal or Turkey Roulades with Dried Apricot Rosemary Stuffing and Apricot Mustard Sauce 
  358.0 cal | 27.0g protein | 17.0g fat

LUNCH: Mexican Chocolate Tart with Cinnamon-Spiced Pecans 
  618.0 cal | 5.0g protein | 47.0g fat

DINNER: Oysters Rockefeller "Deconstructed" 
  867.0 cal | 23.0g protein | 82.0g fat

Day 2 | Score: 0.772
Total: 1914 cal | 74.0g protein | 103.0g fat
--------------------------------------------------

BREAKFAST: Lighter Than Mom's Tuna-Noodle Casserole 
  306.0 cal | 21.0g protein | 4.0g fat

LUNCH: Whitefish Salad and Roasted Beet Coleslaw 
  527.0 cal | 28.0g protein | 29.0g fat

DINNER: Panfried Black Bean Coriander Cakes 
  1081.0 cal | 25.0g protein | 70.0g fat

Day 3 | Score: 0.718
Total: 1808 cal | 75.0g protein | 162.0g fat
--------------------------------------------------

BREAKFAST: Grilled